In [28]:
from kgforge.core import KnowledgeGraphForge, Resource

In [29]:
import getpass
TOKEN = getpass.getpass()

 ········


In [30]:
BUCKET = "dke/kgforge"

forge = KnowledgeGraphForge(
    "../use-cases/prod-forge-nexus.yml",
    bucket=BUCKET,
    token=TOKEN
)

Show-cases options for retrieval.
- The identifiers used are
    - An `id` from the bucket of the configured forge instance
    - A `_self` from the bucket of the configured forge instance
    - An id from a bucket other than the one of the configured forge instance
    - A `_self` from a bucket other than the one of the configured forge instance
- For either `retrieve_source = True` or `retrieve_source = False`
- For `cross_bucket = True` or `cross_bucket = False`

We expect that the following calls with fail:
- An `id` from a bucket other than the one of the configured forge instance, with `cross_bucket = False`, with `retrieve_source = True` or `retrieve_source = False`
- An `_self` from a bucket other than the one of the configured forge instance, with `cross_bucket = False`, with `retrieve_source = True` or `retrieve_source = False`

In the case of the usage of `_self`, it is more clear that user input is the issue because the `_self` holds the bucket where the resource that is attempted to be retrieved is located.
In the case of the usage of `id`, it is not possible to provide a meaningful error as to why the `id` could not be found in the forge instance's configured bucket. 

In [31]:
id_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff"
self_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/https:%2F%2Fbbp.epfl.ch%2Fnexus%2Fv1%2Fresources%2Fdke%2Fkgforge%2F_%2F20fbc97a-fb26-43ff-8093-9136aab25dff"

id_other_bucket = "http://purl.obolibrary.org/obo/GO_0038048"
self_other_bucket = "https://bbp.epfl.ch/nexus/v1/resources/neurosciencegraph/datamodels/_/http:%2F%2Fpurl.obolibrary.org%2Fobo%2FGO_0038048"

all_ret = [
    (id_, "id", "same bucket"), 
    (self_, "self", "same bucket"), 
    (id_other_bucket, "id", "other bucket"),
    (self_other_bucket, "self", "other bucket"),
]

i = 0

for rs in [True, False]:
    for cb in [True, False]:
        for (el, type_, loc) in all_ret:

            print(f"Execution {i}", " using: ", type_, "inside: ", loc, " - Cross bucket:", cb, " - Retrieve source:", rs)

            print(el)
            e = forge.retrieve(el, cross_bucket=cb, retrieve_source=rs)

            if not e:
                print("Not found")
            else:
                print("rev", e._store_metadata._rev, e.context, e.id)
            print("______________________")
            i += 1


Execution 0  using:  id inside:  same bucket  - Cross bucket: True  - Retrieve source: True
https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff
rev 3 https://bbp.neuroshapes.org https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff
______________________
Execution 1  using:  self inside:  same bucket  - Cross bucket: True  - Retrieve source: True
https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/https:%2F%2Fbbp.epfl.ch%2Fnexus%2Fv1%2Fresources%2Fdke%2Fkgforge%2F_%2F20fbc97a-fb26-43ff-8093-9136aab25dff
rev 3 https://bbp.neuroshapes.org https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff
______________________
Execution 2  using:  id inside:  other bucket  - Cross bucket: True  - Retrieve source: True
http://purl.obolibrary.org/obo/GO_0038048
rev 3 https://bbp.neuroshapes.org http://purl.obolibrary.org/obo/GO_0038048
______________________
Execution 3  using:  self inside:  

# Batch Retrieval

In [32]:
import pstats
import cProfile
import json

In [33]:
entities = forge.search({"type": "Entity"})

In [34]:
with cProfile.Profile() as pr:
    test_1 = forge.retrieve([e.id for e in entities])
    pstats.Stats(pr).print_stats(1)
    print([r.get_identifier() if isinstance(r, Resource) else None for r in test_1])
    print("Errors")
    print([r if not isinstance(r, Resource) else None for r in test_1])
    

         5351677 function calls (5054656 primitive calls) in 1.886 seconds

   Random listing order was used
   List reduced from 684 to 1 due to restriction <1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)


['https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/545eaa9a-be49-470f-b5bc-907fbee0cde6', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/9842b6d6-4536-4f27-b2eb-ac2f9363f3f4', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/75549784-55d3-4cec-8968-a00f2a07a2fd', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/a8722f9e-a8a4-499a-b64c-3856ebfd01a6', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d0d38bab-5e01-4d8f-a98a-ccaca6f73b43', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/085d634e-50fd-4fde-9372-6535b9090178', 'https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge

In [35]:
with cProfile.Profile() as pr:
    test_2 = [forge.retrieve(e.id, cross_bucket=True) for e in entities]
    pstats.Stats(pr).print_stats(1)
    print([r.get_identifier() if r else None for r in test_2])

<action> catch_http_error
<error> RetrievalError: 404, message='Not Found', url=URL('https://bbp.epfl.ch/nexus/v1/resolvers/dke/kgforge/_/:%2F%2FNCBITaxon:10090')

         5364936 function calls (5083259 primitive calls) in 8.307 seconds

   Random listing order was used
   List reduced from 694 to 1 due to restriction <1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)


['https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/545eaa9a-be49-470f-b5bc-907fbee0cde6', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/9842b6d6-4536-4f27-b2eb-ac2f9363f3f4', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/75549784-55d3-4cec-8968-a00f2a07a2fd', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/a8722f9e-a8a4-499a-b64c-3856ebfd01a6', 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d0d38bab-5e01-4d8f-a98a-ccac

In [36]:
assert all(
    json.dumps(forge.as_json(e)) == json.dumps(forge.as_json(e2)) 
    for e, e2 in zip(test_1, test_2)
    if e is not None and e2 is not None
)

## Error handling

In [37]:
ids = [e.id for e in entities[:2]] + ["unknown_id"]

In [38]:
test_3 = forge.retrieve(ids)

In [39]:
for r in test_3:
    if isinstance(r, Resource):
        print("Succeeded: ", r._last_action.succeeded, "- Identifier:", r.get_identifier())
    else:
        print("Succeeded: ", r.succeeded)

Succeeded:  True - Identifier: https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/545eaa9a-be49-470f-b5bc-907fbee0cde6
Succeeded:  True - Identifier: https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/9842b6d6-4536-4f27-b2eb-ac2f9363f3f4
Succeeded:  False
